### This notebook can be used to convert data scraped from SCOPUS APIs in .json format into excel spreadsheets

### Import packages

In [2]:
import pandas as pd 
import numpy as np
import json
import os

### Define a recursive function that unpacks the contents of a nested dictionary

In [1]:
def unpack_dic1(item, idd, c,var,flag,out):

    # if the item sent in is a dictionary
    if(isinstance(item,dict)):

        # get keys of the dictionary
        key = item.keys()
        
        for each in key:

            #get the item at each key
            dic1 = item[each]

            if flag == 0:
                # if item is coming from a dictionary
                var1 = idd+var+each+'__'
            else:
                # if item is coming from a list
                var1 = idd+var+each+'__'+str(c)+'__'

            # unpack the dic1 further
            unpack_dic1(dic1,'',0,var1,0,out)

    # if the item sent in is a list
    elif(isinstance(item,list)):
        count = 0
        for each in item:
            # unpack each item in the list
            unpack_dic1(each,'',count,var,1,out)
            count+=1

    # if item is neither a list not a dictionary, then the item is a variable that contains a single value
    else:
        # return it through a dictionary
        dic_out[var]=item
        return dic_out

### Enter the json file and use the above function to get an excel file

In [ ]:
# get location where json files are saved
orig = 'C:/Users/nikhi/Dropbox/Migration and Science/authors publication records/auids_new/'
dir = os.listdir(orig)

# flag to check if the file has already been converted
flag = 0

# get a list of all the excel files in the folder of converted files
fnames = os.listdir('C:/Users/nikhi/Dropbox/Migration and Science/Json_to_excel')

error_auid = []

for folder in dir:

    # get list of all the json files in that folder
    arr = os.listdir(orig + '/' + folder)

    for auid in arr:
        # within each json file

        if flag == 0:
        # if file is in the folder
            fname = folder + '_data_' + auid + '.xlsx'
            if fname not in fnames:
            # if file is not in folder
                flag = 1

        if flag ==1:
        # if file is not in folder
            print(auid, folder)

            # all contents of the file will be saved as a single dictionary
            final_dic = {}
            count = 0

            str1=""
            # define path of the json file to read it
            path = orig + '/' + folder + '/' + auid
            if 'new' not in path:
                # open and read the json file
                json_file = open(path, 'r', encoding='UTF-8')
                str1 = json_file.read()

            c = 0
            if (not(str1=="")):
            # if the file is not empty
                for dct in eval(str1):
                # for each dictionary in the json file
                    try:
                        # get keys of each dictionary
                        k = dct.keys()
                        dic_out = {}

                        for id in k:
                        # for each key
                            try:
                                # unpack the dictionary
                                unpack_dic1(dct[id], id+'__', c, '', 0, {})
                                c += 1

                                # save the dictionary ouput generated from the function
                                final_dic[count]=dic_out
                                final_dic[count]['auid'] = auid
                            except TypeError:
                                error_auid.append(auid)
                    except:
                        error_auid.append(auid)
                    count += 1

                # save the dictionary from each json file into a datafrma and subsequently into an excel file
                df_data=pd.DataFrame.from_dict(final_dic)
                df_data.to_excel(folder + '_data_' + auid + '.xlsx', engine='xlsxwriter')
            else:
            # if file is empty, check again if next file is not in folder
                flag = 0